In [1]:
# InO to OOO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.7533227482609632
Testing R²: 0.7242447776604461
Training Mape: 0.12840343356897382
Testing Mape: 0.13617713282189672
chunk_size: 2000000000
Training R²: 0.8252557268611814
Testing R²: 0.8095724195358404
Training Mape: 0.09877155831189298
Testing Mape: 0.10628183676494485
chunk_size: 3000000000
Training R²: 0.8823023641368511
Testing R²: 0.8568119820578944
Training Mape: 0.07654288500558365
Testing Mape: 0.08110601996105317
chunk_size: 4000000000
Training R²: 0.8975520179746324
Testing R²: 0.8800112370079545
Training Mape: 0.06991489881555085
Testing Mape: 0.07383344574194714
chunk_size: 5000000000
Training R²: 0.9174818388537153
Testing R²: 0.8821351920877867
Training Mape: 0.061673152853304554
Testing Mape: 0.07389000230555437
chunk_size: 10000000000
Training R²: 0.9554409641385191
Testing R²: 0.9235300306123675
Training Mape: 0.043799851855081874
Testing Mape: 0.052739482219395645
chunk_size: 20000000000
Training R²: 0.9778157303280457
Testing R²

In [1]:
# InO to desktop
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.6910936776497234
Testing R²: 0.676544032437079
Training Mape: 0.18409765343390178
Testing Mape: 0.18827471751145444
chunk_size: 2000000000
Training R²: 0.7334777551503983
Testing R²: 0.7116491806041958
Training Mape: 0.16956938055464565
Testing Mape: 0.17574670110841292
chunk_size: 3000000000
Training R²: 0.7691899196414684
Testing R²: 0.7253385632903826
Training Mape: 0.15693529380391408
Testing Mape: 0.16579425716497895
chunk_size: 4000000000
Training R²: 0.7911504404260685
Testing R²: 0.7373325624273959
Training Mape: 0.1496092085845062
Testing Mape: 0.15852020278239326
chunk_size: 5000000000
Training R²: 0.8104705144392689
Testing R²: 0.7423268527749864
Training Mape: 0.14310593532922053
Testing Mape: 0.15626885326538387
chunk_size: 10000000000
Training R²: 0.8552418596469199
Testing R²: 0.7412774828287119
Training Mape: 0.12337849397383734
Testing Mape: 0.1524415356366756
chunk_size: 20000000000
Training R²: 0.9066239559301295
Testing R²: 0.75

In [2]:
# InO to server
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.718348189899501
Testing R²: 0.6952376476099033
Training Mape: 0.12606824581706225
Testing Mape: 0.13188237630504226
chunk_size: 2000000000
Training R²: 0.7357130924170849
Testing R²: 0.6806544568376339
Training Mape: 0.1229320566653013
Testing Mape: 0.13156668151536197
chunk_size: 3000000000
Training R²: 0.7361964015335656
Testing R²: 0.6778301537053854
Training Mape: 0.12182921244344638
Testing Mape: 0.13986451496236743
chunk_size: 4000000000
Training R²: 0.7390141743447367
Testing R²: 0.6901461316561361
Training Mape: 0.12234282914609654
Testing Mape: 0.12995282897021443
chunk_size: 5000000000
Training R²: 0.7363375137614889
Testing R²: 0.6773145948932816
Training Mape: 0.12431023543714013
Testing Mape: 0.1377892985960329
chunk_size: 10000000000
Training R²: 0.7784855385652182
Testing R²: 0.6868007364548734
Training Mape: 0.1105535353260399
Testing Mape: 0.1424460180321974
chunk_size: 20000000000
Training R²: 0.8519210514286037
Testing R²: 0.6412

In [4]:
# OOO to InO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.8006280152860468
Testing R²: 0.7861053859888825
Training Mape: 0.19220372172185204
Testing Mape: 0.20181693872517956
chunk_size: 2000000000
Training R²: 0.8535109133588694
Testing R²: 0.8448451249776255
Training Mape: 0.15115970345113244
Testing Mape: 0.1547619079385732
chunk_size: 3000000000
Training R²: 0.8880825728773998
Testing R²: 0.8552866822524878
Training Mape: 0.12230895660980581
Testing Mape: 0.13276253904181154
chunk_size: 4000000000
Training R²: 0.8980063422679573
Testing R²: 0.8819279820671895
Training Mape: 0.1096437852427383
Testing Mape: 0.1188755959967791
chunk_size: 5000000000
Training R²: 0.9135959978131359
Testing R²: 0.8719190949672951
Training Mape: 0.09604390456621083
Testing Mape: 0.11719668842036472
chunk_size: 10000000000
Training R²: 0.9459728554942582
Testing R²: 0.9006623760963973
Training Mape: 0.07689046527230946
Testing Mape: 0.09851635437968805
chunk_size: 20000000000
Training R²: 0.9747531219605597
Testing R²: 0.92

In [3]:
# OOO to desktop
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_554',
]
chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.7214008833550598
Testing R²: 0.7328732928901014
Training Mape: 0.17854106878791678
Testing Mape: 0.17830975596372572
chunk_size: 2000000000
Training R²: 0.7701954160503407
Testing R²: 0.7437475706780319
Training Mape: 0.15844706034257766
Testing Mape: 0.16394299009503543
chunk_size: 3000000000
Training R²: 0.8021068497289406
Testing R²: 0.7532001761736778
Training Mape: 0.14646108257078322
Testing Mape: 0.15759741845277786
chunk_size: 4000000000
Training R²: 0.8242457775012503
Testing R²: 0.7596349908409283
Training Mape: 0.1389264012628872
Testing Mape: 0.1489718772541412
chunk_size: 5000000000
Training R²: 0.8357844428213137
Testing R²: 0.7998400168332092
Training Mape: 0.12870021713516358
Testing Mape: 0.13869140341093383
chunk_size: 10000000000
Training R²: 0.8755763658350714
Testing R²: 0.7833353961641173
Training Mape: 0.11182008281002652
Testing Mape: 0.1410213072499318
chunk_size: 20000000000
Training R²: 0.9231134877520144
Testing R²: 0.76

In [4]:
# OOO to server
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.6955937142674178
Testing R²: 0.6718011023200388
Training Mape: 0.13741153325712407
Testing Mape: 0.14357817964034783
chunk_size: 2000000000
Training R²: 0.7697282953422072
Testing R²: 0.7209491874015428
Training Mape: 0.1137195587583317
Testing Mape: 0.12128566001722259
chunk_size: 3000000000
Training R²: 0.7693562229308479
Testing R²: 0.7014448850013517
Training Mape: 0.1146418944869485
Testing Mape: 0.13213681808710911
chunk_size: 4000000000
Training R²: 0.7696004086013086
Testing R²: 0.7182434413499015
Training Mape: 0.11526412749772606
Testing Mape: 0.12392094508375646
chunk_size: 5000000000
Training R²: 0.789719711473349
Testing R²: 0.7159114257101846
Training Mape: 0.10865194665569117
Testing Mape: 0.12983627773987552
chunk_size: 10000000000
Training R²: 0.8126481418715242
Testing R²: 0.7172885031113654
Training Mape: 0.10085424032092787
Testing Mape: 0.13321944684710885
chunk_size: 20000000000
Training R²: 0.8542135415482579
Testing R²: 0.66

In [5]:
# desktop to server
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'],
                row['br_pred'],
                row['br_mis_pred'],
                row['l1d_cache_rd'],
                row['l1d_cache_wr'],
                row['l1d_cache'],
                row['l1i_cache'],
                row['l1i_cache_refill'],
                row['context-switches'],
                row['l2d_cache_rd'],
                row['l2d_cache_wr'],
                row['l2d_cache'],
                row['l1d_tlb'],
                row['l1d_tlb_refill_rd'],
                row['l1d_tlb_refill_wr'],
                row['dtlb_walk'],
                row['itlb_walk'],
                row['page-faults'],
                row['l2d_tlb_access'],
                row['l2i_tlb_access'],
                row['l1i_tlb_refill'],
                row['iTLB-loads'],
                row['iTLB-load-misses'],
                row['branch-loads'],
                row['dTLB-loads'],
                row['dTLB-load-misses'],
                row['branch-load-misses'],
                row['vfp_spec'],
                row['inst_spec'],
                row['ase_spec'],
                row['bx_stall'],
                row['decode_stall'],
                row['dispatch_stall'],
                row['fx_stall'],
                row['ixa_stall'],
                row['ixb_stall'],
                row['lx_stall'],
                row['sx_stall'],
                row['bus_access'],
                row['mem_access'],
                row['mem_access_rd'],
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_3.0GHz_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles',
            'br_pred',
            'br_mis_pred',
            'l1d_cache_rd',
            'l1d_cache_wr',
            'l1d_cache',
            'l1i_cache',
            'l1i_cache_refill',
            'context-switches',
            'l2d_cache_rd',
            'l2d_cache_wr',
            'l2d_cache',
            'l1d_tlb',
            'l1d_tlb_refill_rd',
            'l1d_tlb_refill_wr',
            'dtlb_walk',
            'itlb_walk',
            'page-faults',
            'l2d_tlb_access',
            'l2i_tlb_access',
            'l1i_tlb_refill',
            'iTLB-loads',
            'iTLB-load-misses',
            'branch-loads',
            'dTLB-loads',
            'dTLB-load-misses',
            'branch-load-misses',
            'vfp_spec',
            'inst_spec',
            'ase_spec',
            'bx_stall',
            'decode_stall',
            'dispatch_stall',
            'fx_stall',
            'ixa_stall',
            'ixb_stall',
            'lx_stall',
            'sx_stall',
            'bus_access',
            'mem_access',
            'mem_access_rd',
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.727211322573146
Testing R²: 0.6999231595733997
Training Mape: 0.13120868513327424
Testing Mape: 0.1370728327353493
chunk_size: 2000000000
Training R²: 0.7770220044037062
Testing R²: 0.733048992802261
Training Mape: 0.11345889235866469
Testing Mape: 0.11963723569265175
chunk_size: 3000000000
Training R²: 0.7774327327198404
Testing R²: 0.7226523084476628
Training Mape: 0.11270820178765605
Testing Mape: 0.12593479122187268
chunk_size: 4000000000
Training R²: 0.7968335389950065
Testing R²: 0.7308521732641571
Training Mape: 0.10687598395707912
Testing Mape: 0.12147807819305823
chunk_size: 5000000000
Training R²: 0.8229883119623735
Testing R²: 0.7134723894246384
Training Mape: 0.09761511752713292
Testing Mape: 0.12303076278313574
chunk_size: 10000000000
Training R²: 0.8497857971252774
Testing R²: 0.7573345798235489
Training Mape: 0.09068323801040805
Testing Mape: 0.12498022156061978
chunk_size: 20000000000
Training R²: 0.8931701337902705
Testing R²: 0.71

In [8]:
# desktop to InO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'],
                row['br_pred'],
                row['br_mis_pred'],
                row['l1d_cache_rd'],
                row['l1d_cache_wr'],
                row['l1d_cache'],
                row['l1i_cache'],
                row['l1i_cache_refill'],
                row['context-switches'],
                row['l2d_cache_rd'],
                row['l2d_cache_wr'],
                row['l2d_cache'],
                row['l1d_tlb'],
                row['l1d_tlb_refill_rd'],
                row['l1d_tlb_refill_wr'],
                row['dtlb_walk'],
                row['itlb_walk'],
                row['page-faults'],
                row['l2d_tlb_access'],
                row['l2i_tlb_access'],
                row['l1i_tlb_refill'],
                row['iTLB-loads'],
                row['iTLB-load-misses'],
                row['branch-loads'],
                row['dTLB-loads'],
                row['dTLB-load-misses'],
                row['branch-load-misses'],
                row['vfp_spec'],
                row['inst_spec'],
                row['ase_spec'],
                row['bx_stall'],
                row['decode_stall'],
                row['dispatch_stall'],
                row['fx_stall'],
                row['ixa_stall'],
                row['ixb_stall'],
                row['lx_stall'],
                row['sx_stall'],
                row['bus_access'],
                row['mem_access'],
                row['mem_access_rd'],
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles',
            'br_pred',
            'br_mis_pred',
            'l1d_cache_rd',
            'l1d_cache_wr',
            'l1d_cache',
            'l1i_cache',
            'l1i_cache_refill',
            'context-switches',
            'l2d_cache_rd',
            'l2d_cache_wr',
            'l2d_cache',
            'l1d_tlb',
            'l1d_tlb_refill_rd',
            'l1d_tlb_refill_wr',
            'dtlb_walk',
            'itlb_walk',
            'page-faults',
            'l2d_tlb_access',
            'l2i_tlb_access',
            'l1i_tlb_refill',
            'iTLB-loads',
            'iTLB-load-misses',
            'branch-loads',
            'dTLB-loads',
            'dTLB-load-misses',
            'branch-load-misses',
            'vfp_spec',
            'inst_spec',
            'ase_spec',
            'bx_stall',
            'decode_stall',
            'dispatch_stall',
            'fx_stall',
            'ixa_stall',
            'ixb_stall',
            'lx_stall',
            'sx_stall',
            'bus_access',
            'mem_access',
            'mem_access_rd',
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.7608788396226028
Testing R²: 0.7365513401287896
Training Mape: 0.21193932739485133
Testing Mape: 0.2197005709292848
chunk_size: 2000000000
Training R²: 0.7806624681250065
Testing R²: 0.77073735960906
Training Mape: 0.19344178324315714
Testing Mape: 0.1946951174135152
chunk_size: 3000000000
Training R²: 0.7981602063586978
Testing R²: 0.7662139439027111
Training Mape: 0.1830137713215602
Testing Mape: 0.18765029761497082
chunk_size: 4000000000
Training R²: 0.815577660408685
Testing R²: 0.7662222180778089
Training Mape: 0.17183815368736854
Testing Mape: 0.18169142067503358
chunk_size: 5000000000
Training R²: 0.8227272799192334
Testing R²: 0.7762812050201559
Training Mape: 0.1636308958193071
Testing Mape: 0.18631622684601223
chunk_size: 10000000000
Training R²: 0.8528954455977872
Testing R²: 0.7860679234089263
Training Mape: 0.14609409886289138
Testing Mape: 0.17852729906919917
chunk_size: 20000000000
Training R²: 0.8949484145943186
Testing R²: 0.784555

In [9]:
# desktop to OOO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'],
                row['br_pred'],
                row['br_mis_pred'],
                row['l1d_cache_rd'],
                row['l1d_cache_wr'],
                row['l1d_cache'],
                row['l1i_cache'],
                row['l1i_cache_refill'],
                row['context-switches'],
                row['l2d_cache_rd'],
                row['l2d_cache_wr'],
                row['l2d_cache'],
                row['l1d_tlb'],
                row['l1d_tlb_refill_rd'],
                row['l1d_tlb_refill_wr'],
                row['dtlb_walk'],
                row['itlb_walk'],
                row['page-faults'],
                row['l2d_tlb_access'],
                row['l2i_tlb_access'],
                row['l1i_tlb_refill'],
                row['iTLB-loads'],
                row['iTLB-load-misses'],
                row['branch-loads'],
                row['dTLB-loads'],
                row['dTLB-load-misses'],
                row['branch-load-misses'],
                row['vfp_spec'],
                row['inst_spec'],
                row['ase_spec'],
                row['bx_stall'],
                row['decode_stall'],
                row['dispatch_stall'],
                row['fx_stall'],
                row['ixa_stall'],
                row['ixb_stall'],
                row['lx_stall'],
                row['sx_stall'],
                row['bus_access'],
                row['mem_access'],
                row['mem_access_rd'],
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles',
            'br_pred',
            'br_mis_pred',
            'l1d_cache_rd',
            'l1d_cache_wr',
            'l1d_cache',
            'l1i_cache',
            'l1i_cache_refill',
            'context-switches',
            'l2d_cache_rd',
            'l2d_cache_wr',
            'l2d_cache',
            'l1d_tlb',
            'l1d_tlb_refill_rd',
            'l1d_tlb_refill_wr',
            'dtlb_walk',
            'itlb_walk',
            'page-faults',
            'l2d_tlb_access',
            'l2i_tlb_access',
            'l1i_tlb_refill',
            'iTLB-loads',
            'iTLB-load-misses',
            'branch-loads',
            'dTLB-loads',
            'dTLB-load-misses',
            'branch-load-misses',
            'vfp_spec',
            'inst_spec',
            'ase_spec',
            'bx_stall',
            'decode_stall',
            'dispatch_stall',
            'fx_stall',
            'ixa_stall',
            'ixb_stall',
            'lx_stall',
            'sx_stall',
            'bus_access',
            'mem_access',
            'mem_access_rd',
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.7222113355467252
Testing R²: 0.7048806074814544
Training Mape: 0.13721243699680027
Testing Mape: 0.14006001208682833
chunk_size: 2000000000
Training R²: 0.7366127629665724
Testing R²: 0.73249279823532
Training Mape: 0.1313362912851658
Testing Mape: 0.13515558913547593
chunk_size: 3000000000
Training R²: 0.7656155604593871
Testing R²: 0.7453264004335969
Training Mape: 0.12348016404866816
Testing Mape: 0.12156360212145055
chunk_size: 4000000000
Training R²: 0.7909600909021419
Testing R²: 0.7352717198430467
Training Mape: 0.11644727834817763
Testing Mape: 0.12268356630038195
chunk_size: 5000000000
Training R²: 0.797462450340034
Testing R²: 0.7559239161792449
Training Mape: 0.11211626560469953
Testing Mape: 0.12021924311504427
chunk_size: 10000000000
Training R²: 0.8576312998388194
Testing R²: 0.7073427337254444
Training Mape: 0.09339203433142906
Testing Mape: 0.1265684355501373
chunk_size: 20000000000
Training R²: 0.8879247398330529
Testing R²: 0.7681

In [10]:
# server to InO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'], 
                row['br_pred'], 
                row['br_mis_pred'], 
                row['l1d_cache'], 
                row['l1d_tlb'], 
                row['l1d_tlb_rd'], 
                row['l1d_tlb_wr'], 
                row['l2d_cache_rd'], 
                row['l2d_cache_wr'], 
                row['l2d_cache'], 
                row['l1i_cache'], 
                row['l1i_cache_refill'], 
                row['remote_access'], 
                row['dtlb_walk'], 
                row['itlb_walk'], 
                row['l1i_tlb'], 
                row['l2d_tlb'], 
                row['l2d_tlb_rd'], 
                row['l2d_tlb_wr'], 
                row['vfp_spec'], 
                row['inst_spec'], 
                row['ase_spec'], 
                row['stall_backend'], 
                row['stall_frontend'], 
                row['ll_cache_miss_rd'], 
                row['mem_access'], 
                row['mem_access_rd'], 
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles', 
            'br_pred', 
            'br_mis_pred', 
            'l1d_cache', 
            'l1d_tlb', 
            'l1d_tlb_rd', 
            'l1d_tlb_wr', 
            'l2d_cache_rd', 
            'l2d_cache_wr', 
            'l2d_cache', 
            'l1i_cache', 
            'l1i_cache_refill', 
            'remote_access', 
            'dtlb_walk', 
            'itlb_walk', 
            'l1i_tlb', 
            'l2d_tlb', 
            'l2d_tlb_rd', 
            'l2d_tlb_wr', 
            'vfp_spec', 
            'inst_spec', 
            'ase_spec', 
            'stall_backend', 
            'stall_frontend', 
            'll_cache_miss_rd', 
            'mem_access', 
            'mem_access_rd', 
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.7766619350559375
Testing R²: 0.7526961320797898
Training Mape: 0.19822847082735487
Testing Mape: 0.2075689213583482
chunk_size: 2000000000
Training R²: 0.8131491318799707
Testing R²: 0.778099729569258
Training Mape: 0.17806313116063063
Testing Mape: 0.19088055862222772
chunk_size: 3000000000
Training R²: 0.8336577599652779
Testing R²: 0.8092368424336918
Training Mape: 0.16453501709496313
Testing Mape: 0.17420368086376847
chunk_size: 4000000000
Training R²: 0.8494653102013139
Testing R²: 0.805911470986955
Training Mape: 0.15232186306925688
Testing Mape: 0.16648279414446826
chunk_size: 5000000000
Training R²: 0.8735227188210615
Testing R²: 0.7911154943003806
Training Mape: 0.13782874173664436
Testing Mape: 0.16758454297996625
chunk_size: 10000000000
Training R²: 0.9016421676781816
Testing R²: 0.8191406935669961
Training Mape: 0.1196150305920635
Testing Mape: 0.16146744116892342
chunk_size: 20000000000
Training R²: 0.9263319231216207
Testing R²: 0.805

In [11]:
# server to OOO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'], 
                row['br_pred'], 
                row['br_mis_pred'], 
                row['l1d_cache'], 
                row['l1d_tlb'], 
                row['l1d_tlb_rd'], 
                row['l1d_tlb_wr'], 
                row['l2d_cache_rd'], 
                row['l2d_cache_wr'], 
                row['l2d_cache'], 
                row['l1i_cache'], 
                row['l1i_cache_refill'], 
                row['remote_access'], 
                row['dtlb_walk'], 
                row['itlb_walk'], 
                row['l1i_tlb'], 
                row['l2d_tlb'], 
                row['l2d_tlb_rd'], 
                row['l2d_tlb_wr'], 
                row['vfp_spec'], 
                row['inst_spec'], 
                row['ase_spec'], 
                row['stall_backend'], 
                row['stall_frontend'], 
                row['ll_cache_miss_rd'], 
                row['mem_access'], 
                row['mem_access_rd'], 
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles', 
            'br_pred', 
            'br_mis_pred', 
            'l1d_cache', 
            'l1d_tlb', 
            'l1d_tlb_rd', 
            'l1d_tlb_wr', 
            'l2d_cache_rd', 
            'l2d_cache_wr', 
            'l2d_cache', 
            'l1i_cache', 
            'l1i_cache_refill', 
            'remote_access', 
            'dtlb_walk', 
            'itlb_walk', 
            'l1i_tlb', 
            'l2d_tlb', 
            'l2d_tlb_rd', 
            'l2d_tlb_wr', 
            'vfp_spec', 
            'inst_spec', 
            'ase_spec', 
            'stall_backend', 
            'stall_frontend', 
            'll_cache_miss_rd', 
            'mem_access', 
            'mem_access_rd', 
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.7731397865591598
Testing R²: 0.7441179338941282
Training Mape: 0.14769044776695972
Testing Mape: 0.154459504032066
chunk_size: 2000000000
Training R²: 0.782394193828998
Testing R²: 0.7380180388482798
Training Mape: 0.13884475481265512
Testing Mape: 0.15323965566114636
chunk_size: 3000000000
Training R²: 0.8118629466640039
Testing R²: 0.7777999119993249
Training Mape: 0.13291254786001885
Testing Mape: 0.13704152900480202
chunk_size: 4000000000
Training R²: 0.83204482624144
Testing R²: 0.7840143189304456
Training Mape: 0.11968535059986456
Testing Mape: 0.13024344966784876
chunk_size: 5000000000
Training R²: 0.8474109841053103
Testing R²: 0.7984282592714552
Training Mape: 0.11438449848047196
Testing Mape: 0.12244566378123288
chunk_size: 10000000000
Training R²: 0.8834747306791049
Testing R²: 0.7976192823720671
Training Mape: 0.09982522491534683
Testing Mape: 0.12949740085613518
chunk_size: 20000000000
Training R²: 0.9237187691478093
Testing R²: 0.7359

In [6]:
# server to desktop
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'], 
                row['br_pred'], 
                row['br_mis_pred'], 
                row['l1d_cache'], 
                row['l1d_tlb'], 
                row['l1d_tlb_rd'], 
                row['l1d_tlb_wr'], 
                row['l2d_cache_rd'], 
                row['l2d_cache_wr'], 
                row['l2d_cache'], 
                row['l1i_cache'], 
                row['l1i_cache_refill'], 
                row['remote_access'], 
                row['dtlb_walk'], 
                row['itlb_walk'], 
                row['l1i_tlb'], 
                row['l2d_tlb'], 
                row['l2d_tlb_rd'], 
                row['l2d_tlb_wr'], 
                row['vfp_spec'], 
                row['inst_spec'], 
                row['ase_spec'], 
                row['stall_backend'], 
                row['stall_frontend'], 
                row['ll_cache_miss_rd'], 
                row['mem_access'], 
                row['mem_access_rd'], 
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_3.0GHz_554',
]

chunks = [1000000000,2000000000,3000000000,4000000000,5000000000,10000000000,20000000000,25000000000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles', 
            'br_pred', 
            'br_mis_pred', 
            'l1d_cache', 
            'l1d_tlb', 
            'l1d_tlb_rd', 
            'l1d_tlb_wr', 
            'l2d_cache_rd', 
            'l2d_cache_wr', 
            'l2d_cache', 
            'l1i_cache', 
            'l1i_cache_refill', 
            'remote_access', 
            'dtlb_walk', 
            'itlb_walk', 
            'l1i_tlb', 
            'l2d_tlb', 
            'l2d_tlb_rd', 
            'l2d_tlb_wr', 
            'vfp_spec', 
            'inst_spec', 
            'ase_spec', 
            'stall_backend', 
            'stall_frontend', 
            'll_cache_miss_rd', 
            'mem_access', 
            'mem_access_rd', 
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 1000000000
Training R²: 0.705489363697261
Testing R²: 0.672208745532567
Training Mape: 0.15984310370787497
Testing Mape: 0.16386329557480814
chunk_size: 2000000000
Training R²: 0.748091002508475
Testing R²: 0.6756074207767131
Training Mape: 0.13765617252380638
Testing Mape: 0.15984092625906388
chunk_size: 3000000000
Training R²: 0.7678786955699797
Testing R²: 0.7288229619799038
Training Mape: 0.13225171045409453
Testing Mape: 0.14034473492557836
chunk_size: 4000000000
Training R²: 0.786664110502733
Testing R²: 0.7129227853965467
Training Mape: 0.1279202775882085
Testing Mape: 0.14177361329075058
chunk_size: 5000000000
Training R²: 0.7981954393814171
Testing R²: 0.7109333033344922
Training Mape: 0.12293347757751975
Testing Mape: 0.1331818222234514
chunk_size: 10000000000
Training R²: 0.8573813907347163
Testing R²: 0.7258259850828868
Training Mape: 0.10497281137915611
Testing Mape: 0.13580802706806483
chunk_size: 20000000000
Training R²: 0.9000861252692676
Testing R²: 0.74813